In [33]:
import Bio.AlignIO
import Bio.Alphabet
import numpy as np

import gzip

In [48]:
f = gzip.open('../input/Pfam-A.seed.gz', 'rt')

In [49]:
NUM_ALIGNMENTS = 100000

def gen(f):
    g = Bio.AlignIO.parse(f, 'stockholm')
    i = 0
    while True:
        try:
            yield i, next(g)
        except UnicodeDecodeError:
            pass
        except StopIteration:
            break
        i += 1

alignments = []
for i, alignment in gen(f):
    alignments.append(alignment)
    if i >= NUM_ALIGNMENTS:
        break

In [46]:
len(alignments)

8146

In [4]:
def aln_to_arrs(aln):
    a = np.array([rec.seq.upper() for rec in aln])
    
    # Takes a bit of effort to do the alignment position indexing entirely in numpy, 
    # but since we'll be spitting through gigabytes of alignment data later on it's good 
    # if we spend as little time in Python code as possible.
    non_gap_mask = (a != '-')
    non_gap_count = non_gap_mask.sum(axis=1)
    seq_idxs = np.tile(np.arange(non_gap_count.max()), (non_gap_count.shape[0], 1))
    flat_seq_idxs = np.extract(seq_idxs < np.expand_dims(non_gap_count, axis=1), seq_idxs)
    b = np.zeros(a.shape, dtype=np.int)
    b.fill(-1)
    np.place(b, non_gap_mask, flat_seq_idxs)
    
    return (a, b)

In [67]:
gappy_alignments = []
for alignment in alignments:
    symbols, indices = aln_to_arrs(alignment)
    total_positions = np.ones_like(indices).sum()
    num_gaps = (indices == -1).sum()
    if (num_gaps / total_positions) > 0.25:
        gappy_alignments.append((alignments, symbols, indices))
    

In [68]:
len(gappy_alignments)

2321

In [73]:
gappy_alignments[0][2][:60, 50:]

array([[35, 36, 37, 38, 39],
       [39, 40, 41, 42, 43],
       [35, 36, 37, 38, 39],
       [39, 40, 41, 42, 43],
       [35, 36, 37, 38, 39],
       [35, 36, 37, 38, 39],
       [35, 36, 37, 38, 39],
       [29, 30, 31, 32, 33],
       [33, 34, 35, 36, 37],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [31, 32, 33, 34, 35],
       [32, 33, 34, 35, 36],
       [31, 32, 33, 34, 35],
       [33, 34, 35, 36, 37],
       [32, 33, 34, 35, 36],
       [31, 32, 33, 34, 35],
       [33, 34, 35, 36, 37],
       [37, 38, 39, 40, 41],
       [36, 37, 38, 39, 40],
       [29, 30, 31, 32, 33],
       [32, 33, 34, 35, 36],
       [31, 32, 33, 34, 35],
       [39, 40, 41, 42, 43],
       [35, 36, 37, 38, 39],
       [35, 36, 37, 38, 39],
       [35, 36, 37, 38, 39],
       [35, 36, 37, 38, 39],
       [35, 36, -1, -1, 37],
       [35, 36, 37, 38, 39],
       [35, 36